In [2]:
library(feather)

In [3]:
X <- read_feather("../data/170605_preprocessing.feather")
X[1:5,1:5]

Warning message:
“Coercing int64 to double”

age,age_over_90,gender,bmi,bmi_missing
0.10731277,0,0,0.002855699,0
0.07349061,0,1,0.142778950,0
0.27824055,0,0,-0.099502083,0
0.08745643,0,0,0.181029175,0
0.19488550,0,0,-0.078058425,0


R references: [here](https://stats.idre.ucla.edu/r/dae/logit-regression/) and [here](http://data.princeton.edu/R/glms.html). 

# Covariates

In [4]:
covariates <- glm(outcome ~
    + age + age_over_90 
    + gender 
    + bmi + bmi_missing 
    + mech_vent + CHF 
    + mdrd + creatinine_missing + apsiii 
    + micu + on_vaso
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(covariates)


Call:
glm(formula = outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3091  -0.7563  -0.5631   0.8937   2.3308  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.15776    0.20069 -10.752  < 2e-16 ***
age                 1.61180    0.35901   4.489 7.14e-06 ***
age_over_90         1.20725    0.21518   5.610 2.02e-08 ***
gender             -0.02757    0.10470  -0.263  0.79226    
bmi                 0.17098    0.14316   1.194  0.23238    
bmi_missing         0.35909    0.18916   1.898  0.05765 .  
mech_vent           0.35991    0.11060   3.254  0.00114 ** 
CHF                 0.01881    0.11224   0.168  0.86689    
mdrd                0.02376    0.05734   0.414  0.67855    
creatinine_missing  0.07755    0.58275   0.133  0.89414    
apsi

Backward selection to find minimal covariates

In [5]:
minimal <- step(covariates, direction = 'back')
minimal$anova
summary(minimal)

Start:  AIC=2270.08
outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso

                     Df Deviance    AIC
- creatinine_missing  1   2244.1 2268.1
- CHF                 1   2244.1 2268.1
- gender              1   2244.2 2268.2
- mdrd                1   2244.2 2268.2
- micu                1   2245.8 2269.8
<none>                    2244.1 2270.1
- bmi_missing         1   2247.6 2271.6
- bmi                 1   2250.5 2274.5
- mech_vent           1   2254.7 2278.7
- age                 1   2264.9 2288.9
- age_over_90         1   2274.9 2298.9
- on_vaso             1   2299.6 2323.6
- apsiii              1   2351.6 2375.6

Step:  AIC=2268.1
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + apsiii + micu + on_vaso

              Df Deviance    AIC
- CHF          1   2244.1 2266.1
- gender       1   2244.2 2266.2
- mdrd         1   2244.3 2266.3
- micu         1   

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2156,2244.078,2270.078
- creatinine_missing,1,0.01746393,2157,2244.096,2268.096
- CHF,1,0.02830228,2158,2244.124,2266.124
- gender,1,0.06528606,2159,2244.189,2264.189
- mdrd,1,0.15950625,2160,2244.349,2262.349
- micu,1,1.76824967,2161,2246.117,2262.117



Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2906  -0.7552  -0.5650   0.8913   2.3606  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.98884    0.14247 -13.960  < 2e-16 ***
age          1.59960    0.34332   4.659 3.17e-06 ***
age_over_90  1.19794    0.20725   5.780 7.46e-09 ***
bmi          0.17458    0.14762   1.183  0.23696    
bmi_missing  0.36632    0.18882   1.940  0.05237 .  
mech_vent    0.34316    0.10846   3.164  0.00156 ** 
apsiii       0.60458    0.05685  10.634  < 2e-16 ***
on_vaso      0.93711    0.12440   7.533 4.96e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2581.0  on 2168  degrees of freedom
Residual deviance: 2246.1  on 2161  degrees of freedom
AI

# LV

## Fully adjusted

### Fluid day 1

#### Second order

In [7]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
        + lv_hyper * fluid_day1
        + lv_mild * fluid_day1
        + lv_depressed * fluid_day1 
        + lv_severe * fluid_day1 
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + lv_hyper:fluid_day1 + 
    lv_mild:fluid_day1 + lv_depressed:fluid_day1 + lv_severe:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3961  -0.7500  -0.5519   0.8572   2.4520  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.249276   0.205975 -10.920  < 2e-16 ***
age                      1.677036   0.363767   4.610 4.02e-06 ***
age_over_90              1.228859   0.217722   5.644 1.66e-08 ***
gender                  -0.043368   0.106451  -0.407 0.683717    
bmi                      0.167613   0.131824   1.271 0.203554    
bmi_missing              0.360355   0.191140   1.885 0.059390 .  
m

#### Third order

In [71]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
        + lv_hyper * fluid_day1 * micu
        + lv_mild * fluid_day1 * micu
        + lv_depressed * fluid_day1 * micu
        + lv_severe * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + lv_hyper:fluid_day1 + 
    micu:lv_hyper + micu:fluid_day1 + lv_mild:fluid_day1 + micu:lv_mild + 
    lv_depressed:fluid_day1 + micu:lv_depressed + lv_severe:fluid_day1 + 
    micu:lv_severe + micu:lv_hyper:fluid_day1 + micu:lv_mild:fluid_day1 + 
    micu:lv_depressed:fluid_day1 + micu:lv_severe:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4303  -0.7550  -0.5433   0.8547   2.4183  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.29132    0.22264 -10.292  < 2e-16 ***
age                           1.73597    0.36607   4.742 2.11e-06 ***
age_over_90                   1.24398   

(Intercept)                          age 
                   0.1011327                    5.6744459 
                 age_over_90                       gender 
                   3.4693968                    0.9515486 
                         bmi                  bmi_missing 
                   1.1893854                    1.4180436 
                   mech_vent                          CHF 
                   1.4510266                    0.9708181 
                        mdrd           creatinine_missing 
                   1.0286448                    0.9505570 
                      apsiii                         micu 
                   1.9015403                    1.2375770 
                     on_vaso                   lv_missing 
                   2.5695975                    0.9363784 
                    lv_hyper                      lv_mild 
                   2.0260514                    0.5332159 
                lv_depressed                    lv_severe 
                   1.5445081                    2.3131569 
          fluid_day1_missing                   fluid_day1 
                   1.5769494                    0.7484163 
         lv_hyper:fluid_day1                micu:lv_hyper 
                   0.8715108                    0.8026573 
             micu:fluid_day1           lv_mild:fluid_day1 
                   1.2894539                    4.0500182 
                micu:lv_mild      lv_depressed:fluid_day1 
                   1.3044932                    0.9192342 
           micu:lv_depressed         lv_severe:fluid_day1 
                   0.8050228                    0.9566671 
              micu:lv_severe     micu:lv_hyper:fluid_day1 
                   0.6844197                    1.1895382 
     micu:lv_mild:fluid_day1 micu:lv_depressed:fluid_day1 
                   0.1768995                    1.2754281 
   micu:lv_severe:fluid_day1 
                   1.3404041

### fluid day 2

#### second order

In [72]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_hyper * fluid_day2
        + lv_mild * fluid_day2
        + lv_depressed * fluid_day2
        + lv_severe * fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_hyper:fluid_day2 + 
    lv_mild:fluid_day2 + lv_depressed:fluid_day2 + lv_severe:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3464  -0.7529  -0.5477   0.8537   2.3586  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.28373    0.20778 -10.991  < 2e-16 ***
age                      1.68537    0.36471   4.621 3.82e-06 ***
age_over_90              1.22229    0.21837   5.597 2.18e-08 ***
gender                  -0.04421    0.10665  -0.415 0.678464    
bmi                      0.18556    0.16414   1.130 0.258270    
bmi_missing              0.38598    0.19094   2.021 0.043231 *  
mech_ven

#### Third order

In [73]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_hyper * fluid_day2 * micu
        + lv_mild * fluid_day2 * micu
        + lv_depressed * fluid_day2 * micu
        + lv_severe * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_hyper:fluid_day2 + 
    micu:lv_hyper + micu:fluid_day2 + lv_mild:fluid_day2 + micu:lv_mild + 
    lv_depressed:fluid_day2 + micu:lv_depressed + lv_severe:fluid_day2 + 
    micu:lv_severe + micu:lv_hyper:fluid_day2 + micu:lv_mild:fluid_day2 + 
    micu:lv_depressed:fluid_day2 + micu:lv_severe:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3154  -0.7502  -0.5424   0.8161   2.3351  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.31190    0.22423 -10.311  < 2e-16 ***
age                           1.75457    0.36705   4.780 1.75e-06 ***
age_over_90                   1.23607   

(Intercept)                          age 
                  0.09907273                   5.78097295 
                 age_over_90                       gender 
                  3.44205563                   0.95049083 
                         bmi                  bmi_missing 
                  1.21647585                   1.46720013 
                   mech_vent                          CHF 
                  1.46132690                   0.97296152 
                        mdrd           creatinine_missing 
                  1.03068191                   1.01108212 
                      apsiii                         micu 
                  1.81431764                   1.24257249 
                     on_vaso                   lv_missing 
                  2.46800456                   0.93054361 
                    lv_hyper                      lv_mild 
                  1.95796620                   0.28241354 
                lv_depressed                    lv_severe 
                  1.36720509                   2.46482371 
          fluid_day2_missing                   fluid_day2 
                  1.34474644                   0.79168775 
         lv_hyper:fluid_day2                micu:lv_hyper 
                  1.26882972                   0.84117272 
             micu:fluid_day2           lv_mild:fluid_day2 
                  1.39594522                  10.01476849 
                micu:lv_mild      lv_depressed:fluid_day2 
                  2.47126944                   1.61914107 
           micu:lv_depressed         lv_severe:fluid_day2 
                  0.94884068                   1.08388956 
              micu:lv_severe     micu:lv_hyper:fluid_day2 
                  0.66476176                   0.71623275 
     micu:lv_mild:fluid_day2 micu:lv_depressed:fluid_day2 
                  0.04828891                   0.78825600 
   micu:lv_severe:fluid_day2 
                  1.34544541

### fluid day 3

#### second order

In [14]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_hyper * fluid_day3 
        + lv_mild * fluid_day3 
        + lv_depressed * fluid_day3 
        + lv_severe * fluid_day3 
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + lv_hyper:fluid_day3 + 
    lv_mild:fluid_day3 + lv_depressed:fluid_day3 + lv_severe:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4283  -0.7511  -0.5499   0.8498   2.4438  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.319835   0.211115 -10.988  < 2e-16 ***
age                      1.647076   0.364578   4.518 6.25e-06 ***
age_over_90              1.204521   0.218480   5.513 3.52e-08 ***
gender                  -0.046221   0.106613  -0.434 0.664623    
bmi                      0.202087   0.196768   1.027 0.304404    
bmi_missing              0.412263   0.191216   2.156 0.031083 *  
m

#### Third order

In [77]:
model <- update(covariates, ~ . 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_hyper * fluid_day3 * micu
        + lv_mild * fluid_day3 * micu
        + lv_depressed * fluid_day3 * micu
        + lv_severe * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + lv_hyper:fluid_day3 + 
    micu:lv_hyper + micu:fluid_day3 + lv_mild:fluid_day3 + micu:lv_mild + 
    lv_depressed:fluid_day3 + micu:lv_depressed + lv_severe:fluid_day3 + 
    micu:lv_severe + micu:lv_hyper:fluid_day3 + micu:lv_mild:fluid_day3 + 
    micu:lv_depressed:fluid_day3 + micu:lv_severe:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3287  -0.7480  -0.5444   0.8425   2.4770  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.352452   0.227054 -10.361  < 2e-16 ***
age                           1.710569   0.367006   4.661 3.15e-06 ***
age_over_90                   1.23379

(Intercept)                          age 
                  0.09513564                   5.53210829 
                 age_over_90                       gender 
                  3.43423905                   0.95705202 
                         bmi                  bmi_missing 
                  1.23714466                   1.51274793 
                   mech_vent                          CHF 
                  1.48914648                   0.99815726 
                        mdrd           creatinine_missing 
                  1.03294851                   1.09006656 
                      apsiii                         micu 
                  1.79603141                   1.25273760 
                     on_vaso                   lv_missing 
                  2.41783548                   0.94388201 
                    lv_hyper                      lv_mild 
                  1.95215074                   0.61410590 
                lv_depressed                    lv_severe 
                  1.42760333                   2.47382405 
          fluid_day3_missing                   fluid_day3 
                  1.20471678                   0.73646717 
         lv_hyper:fluid_day3                micu:lv_hyper 
                  1.43010253                   0.83083477 
             micu:fluid_day3           lv_mild:fluid_day3 
                  1.67943409                   4.81240677 
                micu:lv_mild      lv_depressed:fluid_day3 
                  1.14942274                   1.59683753 
           micu:lv_depressed         lv_severe:fluid_day3 
                  0.91125914                   1.87119603 
              micu:lv_severe     micu:lv_hyper:fluid_day3 
                  0.64896259                   0.67936563 
     micu:lv_mild:fluid_day3 micu:lv_depressed:fluid_day3 
                  0.09009223                   0.91723460 
   micu:lv_severe:fluid_day3 
                  0.55450133

## Not adjusted

In [78]:
model <- glm(outcome ~ 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
        + lv_mild * fluid_day1 * micu
        + lv_hyper * fluid_day1 * micu
        + lv_mild * fluid_day1 * micu
        + lv_depressed * fluid_day1 * micu
        + lv_severe * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ +lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + lv_mild * fluid_day1 * 
    micu + lv_hyper * fluid_day1 * micu + lv_mild * fluid_day1 * 
    micu + lv_depressed * fluid_day1 * micu + lv_severe * fluid_day1 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6346  -0.8071  -0.7119   1.2780   2.2906  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.40789    0.15312  -9.195   <2e-16 ***
lv_missing                    0.11214    0.15479   0.724   0.4688    
lv_hyper                      0.77214    0.36884   2.093   0.0363 *  
lv_mild                      -0.51010    0.72848  -0.700   0.4838    
lv_depressed                  0.64988    0.54528   1.192   0.2333    
lv_severe                     0.74339    0.65675   1.132   0.2577    
fluid_day1_missing           -0.

(Intercept)                   lv_missing 
                   0.2446597                    1.1186659 
                    lv_hyper                      lv_mild 
                   2.1643915                    0.6004374 
                lv_depressed                    lv_severe 
                   1.9153160                    2.1030621 
          fluid_day1_missing                   fluid_day1 
                   0.8098216                    1.1108284 
                        micu           lv_mild:fluid_day1 
                   1.4557658                    2.7287881 
                lv_mild:micu              fluid_day1:micu 
                   1.4117493                    1.2231803 
         lv_hyper:fluid_day1                lv_hyper:micu 
                   1.1184661                    0.8229732 
     lv_depressed:fluid_day1            lv_depressed:micu 
                   1.4394112                    0.8590693 
        lv_severe:fluid_day1               lv_severe:micu 
                   0.9662953                    0.9739671 
     lv_mild:fluid_day1:micu     lv_hyper:fluid_day1:micu 
                   0.2752319                    1.0681746 
lv_depressed:fluid_day1:micu    lv_severe:fluid_day1:micu 
                   0.8465488                    1.3890915

In [79]:
model <- glm(outcome ~ 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
        + lv_mild * fluid_day2 * micu
        + lv_hyper * fluid_day2 * micu
        + lv_mild * fluid_day2 * micu
        + lv_depressed * fluid_day2 * micu
        + lv_severe * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ +lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_mild * fluid_day2 * 
    micu + lv_hyper * fluid_day2 * micu + lv_mild * fluid_day2 * 
    micu + lv_depressed * fluid_day2 * micu + lv_severe * fluid_day2 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7293  -0.7982  -0.6886   1.2637   2.2205  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.41466    0.15421  -9.174  < 2e-16 ***
lv_missing                    0.12866    0.15559   0.827  0.40829    
lv_hyper                      0.69142    0.38228   1.809  0.07050 .  
lv_mild                      -1.10773    0.93754  -1.182  0.23740    
lv_depressed                  0.37822    0.63794   0.593  0.55326    
lv_severe                     0.78332    0.59727   1.311  0.18969    
fluid_day2_missing           -0.

(Intercept)                   lv_missing 
                  0.24300708                   1.13729782 
                    lv_hyper                      lv_mild 
                  1.99654770                   0.33030879 
                lv_depressed                    lv_severe 
                  1.45968605                   2.18872468 
          fluid_day2_missing                   fluid_day2 
                  0.93744313                   1.12751114 
                        micu           lv_mild:fluid_day2 
                  1.44778513                   7.58166115 
                lv_mild:micu              fluid_day2:micu 
                  2.45623692                   1.33343808 
         lv_hyper:fluid_day2                lv_hyper:micu 
                  1.50342989                   0.94098562 
     lv_depressed:fluid_day2            lv_depressed:micu 
                  2.24757931                   1.16243004 
        lv_severe:fluid_day2               lv_severe:micu 
                  1.08651543                   0.96749905 
     lv_mild:fluid_day2:micu     lv_hyper:fluid_day2:micu 
                  0.06548446                   0.62152075 
lv_depressed:fluid_day2:micu    lv_severe:fluid_day2:micu 
                  0.50547601                   1.44273045

In [81]:
model <- glm(outcome ~ 
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
        + lv_mild * fluid_day3 * micu
        + lv_hyper * fluid_day3 * micu
        + lv_mild * fluid_day3 * micu
        + lv_depressed * fluid_day3 * micu
        + lv_severe * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ +lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + lv_mild * fluid_day3 * 
    micu + lv_hyper * fluid_day3 * micu + lv_mild * fluid_day3 * 
    micu + lv_depressed * fluid_day3 * micu + lv_severe * fluid_day3 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4888  -0.7906  -0.6834   1.2890   2.0541  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.386369   0.153688  -9.021  < 2e-16 ***
lv_missing                    0.140485   0.155860   0.901  0.36740    
lv_hyper                      0.787317   0.362797   2.170  0.03000 *  
lv_mild                      -0.396294   0.690867  -0.574  0.56623    
lv_depressed                  0.579966   0.569062   1.019  0.30813    
lv_severe                     0.746905   0.578662   1.291  0.19679    
fluid_day3_missing       

(Intercept)                   lv_missing 
                   0.2499812                    1.1508313 
                    lv_hyper                      lv_mild 
                   2.1974931                    0.6728087 
                lv_depressed                    lv_severe 
                   1.7859773                    2.1104577 
          fluid_day3_missing                   fluid_day3 
                   0.8920900                    0.9735400 
                        micu           lv_mild:fluid_day3 
                   1.4294914                    3.9629632 
                lv_mild:micu              fluid_day3:micu 
                   1.2149754                    1.6548985 
         lv_hyper:fluid_day3                lv_hyper:micu 
                   1.3836014                    0.8676357 
     lv_depressed:fluid_day3            lv_depressed:micu 
                   2.1475902                    0.9608553 
        lv_severe:fluid_day3               lv_severe:micu 
                   1.7657252                    1.0057603 
     lv_mild:fluid_day3:micu     lv_hyper:fluid_day3:micu 
                   0.1143488                    0.6529102 
lv_depressed:fluid_day3:micu    lv_severe:fluid_day3:micu 
                   0.6061836                    0.6286506

## Minimally adjusted

In [82]:
model <- update(minimal, ~ . 
        + fluid_day1_missing + lv_missing
        + lv_mild * fluid_day1 * micu 
        + lv_hyper * fluid_day1 * micu
        + lv_mild * fluid_day1 * micu
        + lv_depressed * fluid_day1 * micu
        + lv_severe * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso + fluid_day1_missing + lv_missing + 
    lv_mild + fluid_day1 + micu + lv_hyper + lv_depressed + lv_severe + 
    lv_mild:fluid_day1 + lv_mild:micu + fluid_day1:micu + fluid_day1:lv_hyper + 
    micu:lv_hyper + fluid_day1:lv_depressed + micu:lv_depressed + 
    fluid_day1:lv_severe + micu:lv_severe + lv_mild:fluid_day1:micu + 
    fluid_day1:micu:lv_hyper + fluid_day1:micu:lv_depressed + 
    fluid_day1:micu:lv_severe, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4102  -0.7553  -0.5441   0.8426   2.4561  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.30157    0.21783 -10.566  < 2e-16 ***
age                           1.67300    0.35043   4.774 1.81e-06 ***
age_over_90                   1.20713    0.21082   5.726 1.03e-08 ***
bmi          

(Intercept)                          age 
                   0.1001013                    5.3281378 
                 age_over_90                          bmi 
                   3.3438708                    1.1873350 
                 bmi_missing                    mech_vent 
                   1.4158542                    1.4569222 
                      apsiii                      on_vaso 
                   1.8870425                    2.5533524 
          fluid_day1_missing                   lv_missing 
                   1.5618423                    0.9392906 
                     lv_mild                   fluid_day1 
                   0.5397316                    0.7493308 
                        micu                     lv_hyper 
                   1.2293491                    2.0140736 
                lv_depressed                    lv_severe 
                   1.5215503                    2.2807311 
          lv_mild:fluid_day1                 lv_mild:micu 
                   4.0550788                    1.2861298 
             fluid_day1:micu          fluid_day1:lv_hyper 
                   1.2905237                    0.8728059 
               micu:lv_hyper      fluid_day1:lv_depressed 
                   0.8054218                    0.9316753 
           micu:lv_depressed         fluid_day1:lv_severe 
                   0.8105049                    0.9638534 
              micu:lv_severe      lv_mild:fluid_day1:micu 
                   0.6903154                    0.1761143 
    fluid_day1:micu:lv_hyper fluid_day1:micu:lv_depressed 
                   1.1905935                    1.2566534 
   fluid_day1:micu:lv_severe 
                   1.3302537

In [83]:
model <- update(minimal, ~ . 
        + fluid_day2_missing + lv_missing
        + lv_mild * fluid_day2 * micu 
        + lv_hyper * fluid_day2 * micu
        + lv_mild * fluid_day2 * micu
        + lv_depressed * fluid_day2 * micu
        + lv_severe * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso + fluid_day2_missing + lv_missing + 
    lv_mild + fluid_day2 + micu + lv_hyper + lv_depressed + lv_severe + 
    lv_mild:fluid_day2 + lv_mild:micu + fluid_day2:micu + fluid_day2:lv_hyper + 
    micu:lv_hyper + fluid_day2:lv_depressed + micu:lv_depressed + 
    fluid_day2:lv_severe + micu:lv_severe + lv_mild:fluid_day2:micu + 
    fluid_day2:micu:lv_hyper + fluid_day2:micu:lv_depressed + 
    fluid_day2:micu:lv_severe, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3004  -0.7494  -0.5449   0.8144   2.3478  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.32118    0.21946 -10.577  < 2e-16 ***
age                           1.69140    0.35119   4.816 1.46e-06 ***
age_over_90                   1.19817    0.21157   5.663 1.49e-08 ***
bmi          

(Intercept)                          age 
                  0.09815794                   5.42706482 
                 age_over_90                          bmi 
                  3.31404023                   1.21198030 
                 bmi_missing                    mech_vent 
                  1.46441022                   1.46693147 
                      apsiii                      on_vaso 
                  1.79862378                   2.45246341 
          fluid_day2_missing                   lv_missing 
                  1.33447736                   0.93346028 
                     lv_mild                   fluid_day2 
                  0.28513214                   0.79321797 
                        micu                     lv_hyper 
                  1.23535953                   1.94247170 
                lv_depressed                    lv_severe 
                  1.34347498                   2.44329301 
          lv_mild:fluid_day2                 lv_mild:micu 
                 10.10377142                   2.44297907 
             fluid_day2:micu          fluid_day2:lv_hyper 
                  1.39487268                   1.26853885 
               micu:lv_hyper      fluid_day2:lv_depressed 
                  0.84468582                   1.63383396 
           micu:lv_depressed         fluid_day2:lv_severe 
                  0.95676657                   1.09739851 
              micu:lv_severe      lv_mild:fluid_day2:micu 
                  0.66751475                   0.04804275 
    fluid_day2:micu:lv_hyper fluid_day2:micu:lv_depressed 
                  0.71907009                   0.77779324 
   fluid_day2:micu:lv_severe 
                  1.33179063

In [84]:
model <- update(minimal, ~ . 
        + fluid_day3_missing + lv_missing
        + lv_mild * fluid_day3 * micu 
        + lv_hyper * fluid_day3 * micu
        + lv_mild * fluid_day3 * micu
        + lv_depressed * fluid_day3 * micu
        + lv_severe * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + bmi + bmi_missing + 
    mech_vent + apsiii + on_vaso + fluid_day3_missing + lv_missing + 
    lv_mild + fluid_day3 + micu + lv_hyper + lv_depressed + lv_severe + 
    lv_mild:fluid_day3 + lv_mild:micu + fluid_day3:micu + fluid_day3:lv_hyper + 
    micu:lv_hyper + fluid_day3:lv_depressed + micu:lv_depressed + 
    fluid_day3:lv_severe + micu:lv_severe + lv_mild:fluid_day3:micu + 
    fluid_day3:micu:lv_hyper + fluid_day3:micu:lv_depressed + 
    fluid_day3:micu:lv_severe, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3122  -0.7462  -0.5439   0.8333   2.5135  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -2.36085    0.22237 -10.617  < 2e-16 ***
age                           1.66759    0.35157   4.743 2.10e-06 ***
age_over_90                   1.20528    0.21174   5.692 1.25e-08 ***
bmi          

(Intercept)                          age 
                  0.09433966                   5.29939709 
                 age_over_90                          bmi 
                  3.33767871                   1.23208527 
                 bmi_missing                    mech_vent 
                  1.50996180                   1.49786662 
                      apsiii                      on_vaso 
                  1.77785241                   2.40363379 
          fluid_day3_missing                   lv_missing 
                  1.20114085                   0.94857735 
                     lv_mild                   fluid_day3 
                  0.62041712                   0.73777641 
                        micu                     lv_hyper 
                  1.25156194                   1.93705421 
                lv_depressed                    lv_severe 
                  1.42022518                   2.48045057 
          lv_mild:fluid_day3                 lv_mild:micu 
                  4.94760515                   1.13735152 
             fluid_day3:micu          fluid_day3:lv_hyper 
                  1.67650177                   1.43044784 
               micu:lv_hyper      fluid_day3:lv_depressed 
                  0.83264492                   1.60372465 
           micu:lv_depressed         fluid_day3:lv_severe 
                  0.91419017                   1.87978688 
              micu:lv_severe      lv_mild:fluid_day3:micu 
                  0.64743321                   0.08786992 
    fluid_day3:micu:lv_hyper fluid_day3:micu:lv_depressed 
                  0.68234683                   0.91061592 
   fluid_day3:micu:lv_severe 
                  0.55242891

## Model selection via forward step-wise regression

In [109]:
model <- update(covariates, ~ .
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
scope <- formula(update(model, ~.
    + micu * lv_hyper * fluid_day1 + micu * lv_mild * fluid_day1
    + micu * lv_depressed * fluid_day1 + micu * lv_severe * fluid_day1
    ))
select <- step(model, direction = 'forward', scope = list(upper = scope))
select$anova
summary(select)
exp(coef(select))

select2 <- update(select, ~ . + micu * lv_mild * fluid_day1)
summary(select2)
exp(coef(select2))

select3 <- update(select, ~ . + micu * lv_mild + micu * fluid_day1 + lv_mild * fluid_day1)
summary(select3)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4078  -0.7535  -0.5510   0.8716   2.4622  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.257016   0.205845 -10.965  < 2e-16 ***
age                 1.677811   0.363670   4.614 3.96e-06 ***
age_over_90         1.218829   0.217286   5.609 2.03e-08 ***
gender             -0.040844   0.106182  -0.385 0.700486    
bmi                 0.168666   0.133213   1.266 0.205463    
bmi_missing         0.360649   0.191038   1.888 0.059048 .  
mech_vent           0.376394   0.112458   3.347 0.000817 ***
CHF                -0.026956   0.116823  -0.231 0.817515    
mdrd       

Start:  AIC=2262.19
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day1_missing + fluid_day1

                          Df Deviance    AIC
+ micu:fluid_day1          1   2219.2 2261.2
<none>                         2222.2 2262.2
+ lv_severe:fluid_day1     1   2220.8 2262.8
+ micu:lv_severe           1   2221.6 2263.6
+ lv_depressed:fluid_day1  1   2221.9 2263.9
+ micu:lv_hyper            1   2222.1 2264.1
+ micu:lv_depressed        1   2222.1 2264.1
+ lv_hyper:fluid_day1      1   2222.1 2264.1
+ lv_mild:fluid_day1       1   2222.2 2264.2
+ micu:lv_mild             1   2222.2 2264.2

Step:  AIC=2261.2
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day1_missing 

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2149,2222.185,2262.185
+ micu:fluid_day1,-1,2.988935,2148,2219.196,2261.196



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + micu:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4250  -0.7539  -0.5493   0.8582   2.4786  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.2303914  0.2054956 -10.854  < 2e-16 ***
age                 1.6922152  0.3641140   4.647 3.36e-06 ***
age_over_90         1.2228062  0.2174205   5.624 1.86e-08 ***
gender             -0.0435185  0.1062858  -0.409 0.682211    
bmi                 0.1699862  0.1350485   1.259 0.208137    
bmi_missing         0.3629593  0.1910698   1.900 0.057484 .  
mech_vent           0.3708231  0.1125051   3.296 0.000981 ***
CHF                -0.0198165  0.1169708  -0.1

(Intercept)                age        age_over_90             gender 
         0.1074863          5.4314995          3.3967064          0.9574148 
               bmi        bmi_missing          mech_vent                CHF 
         1.1852885          1.4375773          1.4489268          0.9803785 
              mdrd creatinine_missing             apsiii               micu 
         1.0336094          0.9991149          1.8864933          1.1593029 
           on_vaso         lv_missing           lv_hyper            lv_mild 
         2.5768543          0.9410544          1.7029546          0.7391607 
      lv_depressed          lv_severe fluid_day1_missing         fluid_day1 
         1.2770563          1.6417217          1.5507628          0.7766327 
   micu:fluid_day1 
         1.2740752


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + micu:fluid_day1 + 
    micu:lv_mild + lv_mild:fluid_day1 + micu:lv_mild:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4421  -0.7587  -0.5432   0.8469   2.4178  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.23604    0.20747 -10.777  < 2e-16 ***
age                      1.73633    0.36565   4.749 2.05e-06 ***
age_over_90              1.23126    0.21804   5.647 1.63e-08 ***
gender                  -0.04297    0.10651  -0.403  0.68660    
bmi                      0.17625    0.14610   1.206  0.22767    
bmi_missing              0.35773    0.19137   1.869  0.06159 .  
mech_vent           

(Intercept)                     age             age_over_90 
              0.1068809               5.6764856               3.4255430 
                 gender                     bmi             bmi_missing 
              0.9579358               1.1927360               1.4300746 
              mech_vent                     CHF                    mdrd 
              1.4479043               0.9708780               1.0304095 
     creatinine_missing                  apsiii                    micu 
              0.9869455               1.9022213               1.1542341 
                on_vaso              lv_missing                lv_hyper 
              2.5676544               0.9386333               1.6985133 
                lv_mild            lv_depressed               lv_severe 
              0.5040356               1.2800497               1.6420663 
     fluid_day1_missing              fluid_day1         micu:fluid_day1 
              1.5701372               0.7277924               1.3765402 
           micu:lv_mild      lv_mild:fluid_day1 micu:lv_mild:fluid_day1 
              1.3993424               4.1682643               0.1657722


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day1_missing + fluid_day1 + micu:fluid_day1 + 
    micu:lv_mild + lv_mild:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4249  -0.7547  -0.5499   0.8567   2.4708  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.233975   0.207394 -10.772  < 2e-16 ***
age                 1.691594   0.364115   4.646 3.39e-06 ***
age_over_90         1.223776   0.217499   5.627 1.84e-08 ***
gender             -0.043063   0.106320  -0.405 0.685456    
bmi                 0.170422   0.135798   1.255 0.209491    
bmi_missing         0.362866   0.191225   1.898 0.057751 .  
mech_vent           0.370969   0.112525   3.297 0.000978 ***
CHF          

In [110]:
model <- update(covariates, ~ .
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
scope <- formula(update(model, ~.
    + micu * lv_hyper * fluid_day2 + micu * lv_mild * fluid_day2
    + micu * lv_depressed * fluid_day2 + micu * lv_severe * fluid_day2
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)
exp(coef(select))

select2 <- update(select, ~ . + micu * lv_mild * fluid_day2)
summary(select2)
exp(coef(select2))

select3 <- update(select, ~ . + micu * lv_mild + micu * fluid_day2 + lv_mild * fluid_day2)
summary(select3)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3380  -0.7499  -0.5508   0.8587   2.4629  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.275555   0.207174 -10.984  < 2e-16 ***
age                 1.640919   0.363402   4.515 6.32e-06 ***
age_over_90         1.185495   0.217043   5.462 4.71e-08 ***
gender             -0.044914   0.106250  -0.423  0.67250    
bmi                 0.186310   0.164615   1.132  0.25772    
bmi_missing         0.396283   0.190446   2.081  0.03745 *  
mech_vent           0.376622   0.113441   3.320  0.00090 ***
CHF                -0.006474   0.116941  -0.055  0.95585    
mdrd       

Start:  AIC=2261.47
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day2_missing + fluid_day2

                          Df Deviance    AIC
+ lv_severe:fluid_day2     1   2218.7 2260.7
+ lv_mild:fluid_day2       1   2218.8 2260.8
+ micu:fluid_day2          1   2219.3 2261.3
<none>                         2221.5 2261.5
+ lv_depressed:fluid_day2  1   2219.9 2261.9
+ micu:lv_severe           1   2220.9 2262.9
+ micu:lv_hyper            1   2221.3 2263.3
+ lv_hyper:fluid_day2      1   2221.3 2263.3
+ micu:lv_depressed        1   2221.4 2263.4
+ micu:lv_mild             1   2221.5 2263.5

Step:  AIC=2260.65
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day2_missing

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2149,2221.469,2261.469
+ lv_severe:fluid_day2,-1,2.815893,2148,2218.653,2260.653
+ lv_mild:fluid_day2,-1,2.403386,2147,2216.250,2260.250
+ micu:fluid_day2,-1,2.047402,2146,2214.202,2260.202



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_severe:fluid_day2 + 
    lv_mild:fluid_day2 + micu:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3409  -0.7496  -0.5499   0.8416   2.3322  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -2.24294    0.20746 -10.811  < 2e-16 ***
age                   1.68035    0.36444   4.611 4.01e-06 ***
age_over_90           1.21194    0.21808   5.557 2.74e-08 ***
gender               -0.04203    0.10661  -0.394  0.69338    
bmi                   0.18885    0.16912   1.117  0.26414    
bmi_missing           0.38429    0.19050   2.017  0.04367 *  
mech_vent             0.37394    0.11368   3.289  0.00100 *

(Intercept)                  age          age_over_90 
           0.1061461            5.3674265            3.3599971 
              gender                  bmi          bmi_missing 
           0.9588393            1.2078614            1.4685714 
           mech_vent                  CHF                 mdrd 
           1.4534447            0.9909416            1.0350125 
  creatinine_missing               apsiii                 micu 
           1.0691907            1.8059604            1.1615027 
             on_vaso           lv_missing             lv_hyper 
           2.4770329            0.9420273            1.6884123 
             lv_mild         lv_depressed            lv_severe 
           0.7595550            1.3186567            1.7244348 
  fluid_day2_missing           fluid_day2 lv_severe:fluid_day2 
           1.3240409            0.8907438            1.3954537 
  lv_mild:fluid_day2      micu:fluid_day2 
           0.6485773            1.2412479


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_severe:fluid_day2 + 
    lv_mild:fluid_day2 + micu:fluid_day2 + micu:lv_mild + micu:lv_mild:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3536  -0.7483  -0.5467   0.8137   2.3199  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.24661    0.20914 -10.742  < 2e-16 ***
age                      1.73242    0.36632   4.729 2.25e-06 ***
age_over_90              1.21501    0.21885   5.552 2.83e-08 ***
gender                  -0.04477    0.10688  -0.419  0.67529    
bmi                      0.19869    0.19144   1.038  0.29934    
bmi_missing              0.38180    0.19077   2.001  0.04536 *

(Intercept)                     age             age_over_90 
             0.10575755              5.65434652              3.37032588 
                 gender                     bmi             bmi_missing 
             0.95621357              1.21980100              1.46491426 
              mech_vent                     CHF                    mdrd 
             1.45111048              0.97737913              1.03145034 
     creatinine_missing                  apsiii                    micu 
             1.05865677              1.81593836              1.15611562 
                on_vaso              lv_missing                lv_hyper 
             2.47072331              0.93721996              1.68324032 
                lv_mild            lv_depressed               lv_severe 
             0.26696439              1.32604672              1.72639499 
     fluid_day2_missing              fluid_day2    lv_severe:fluid_day2 
             1.34081378              0.82930893              1.38346733 
     lv_mild:fluid_day2         micu:fluid_day2            micu:lv_mild 
             9.56255444              1.35182491              2.65198445 
micu:lv_mild:fluid_day2 
             0.04993655


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day2_missing + fluid_day2 + lv_severe:fluid_day2 + 
    lv_mild:fluid_day2 + micu:fluid_day2 + micu:lv_mild, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3415  -0.7494  -0.5492   0.8408   2.3408  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -2.249395   0.209333 -10.746  < 2e-16 ***
age                   1.679469   0.364467   4.608 4.07e-06 ***
age_over_90           1.212014   0.218077   5.558 2.73e-08 ***
gender               -0.042096   0.106607  -0.395  0.69294    
bmi                   0.188940   0.169412   1.115  0.26473    
bmi_missing           0.385635   0.190622   2.023  0.04307 *  
mech_vent             0.373569   0.11

In [87]:
model <- update(covariates, ~ .
        + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe 
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * lv_hyper * fluid_day3 + micu * lv_mild * fluid_day3
    + micu * lv_depressed * fluid_day3 + micu * lv_severe * fluid_day3
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)
exp(coef(select))

select2 <- update(select, ~ . + micu * lv_mild * fluid_day2)
summary(select2)
exp(coef(select2))

Start:  AIC=2259.57
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day3_missing + fluid_day3

                          Df Deviance    AIC
+ micu:fluid_day3          1   2215.0 2257.0
+ lv_mild:fluid_day3       1   2216.1 2258.1
+ lv_depressed:fluid_day3  1   2217.0 2259.0
<none>                         2219.6 2259.6
+ micu:lv_severe           1   2219.1 2261.1
+ lv_severe:fluid_day3     1   2219.2 2261.2
+ micu:lv_hyper            1   2219.4 2261.4
+ micu:lv_depressed        1   2219.6 2261.6
+ micu:lv_mild             1   2219.6 2261.6
+ lv_hyper:fluid_day3      1   2219.6 2261.6

Step:  AIC=2257.01
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe + 
    fluid_day3_missing

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2149,2219.568,2259.568
+ micu:fluid_day3,-1,4.555316,2148,2215.013,2257.013
+ lv_mild:fluid_day3,-1,3.758530,2147,2211.254,2255.254
+ lv_depressed:fluid_day3,-1,2.015157,2146,2209.239,2255.239



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + micu:fluid_day3 + 
    lv_mild:fluid_day3 + lv_depressed:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3165  -0.7497  -0.5484   0.8511   2.4486  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.28492    0.21056 -10.851  < 2e-16 ***
age                      1.65707    0.36484   4.542 5.58e-06 ***
age_over_90              1.20998    0.21861   5.535 3.11e-08 ***
gender                  -0.05225    0.10668  -0.490  0.62426    
bmi                      0.20322    0.19744   1.029  0.30334    
bmi_missing              0.42026    0.19104   2.200  0.02782 *  
mech_vent                0.39477   

(Intercept)                     age             age_over_90 
              0.1017825               5.2439098               3.3534246 
                 gender                     bmi             bmi_missing 
              0.9490902               1.2253474               1.5223617 
              mech_vent                     CHF                    mdrd 
              1.4840380               1.0111935               1.0339078 
     creatinine_missing                  apsiii                    micu 
              1.0938997               1.7923040               1.1827563 
                on_vaso              lv_missing                lv_hyper 
              2.4133407               0.9510373               1.6931526 
                lv_mild            lv_depressed               lv_severe 
              0.7468056               1.3142954               1.6732172 
     fluid_day3_missing              fluid_day3         micu:fluid_day3 
              1.1984678               0.8552812               1.4274577 
     lv_mild:fluid_day3 lv_depressed:fluid_day3 
              0.5808319               1.4699135


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + lv_missing + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + fluid_day3_missing + fluid_day3 + fluid_day2 + 
    micu:fluid_day3 + lv_mild:fluid_day3 + lv_depressed:fluid_day3 + 
    micu:lv_mild + micu:fluid_day2 + lv_mild:fluid_day2 + micu:lv_mild:fluid_day2, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3375  -0.7464  -0.5435   0.8382   2.3302  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.290767   0.212927 -10.758  < 2e-16 ***
age                      1.711667   0.367069   4.663 3.12e-06 ***
age_over_90              1.218574   0.219964   5.540 3.03e-08 ***
gender                  -0.053169   0.106992  -0.497 0.619228    
bmi                      0.212886   0.222698   0.956 0.339103 

(Intercept)                     age             age_over_90 
             0.10118878              5.53818439              3.38236190 
                 gender                     bmi             bmi_missing 
             0.94821951              1.23724370              1.51009642 
              mech_vent                     CHF                    mdrd 
             1.49167292              0.99628511              1.03117583 
     creatinine_missing                  apsiii                    micu 
             1.06213141              1.81375707              1.17757512 
                on_vaso              lv_missing                lv_hyper 
             2.41969603              0.95109285              1.69467154 
                lv_mild            lv_depressed               lv_severe 
             0.25137544              1.31944029              1.67995254 
     fluid_day3_missing              fluid_day3              fluid_day2 
             1.17734025              0.81922944              0.97048045 
        micu:fluid_day3      lv_mild:fluid_day3 lv_depressed:fluid_day3 
             1.64350408              0.61778095              1.46391570 
           micu:lv_mild         micu:fluid_day2      lv_mild:fluid_day2 
             2.83675549              0.93769584             13.45606281 
micu:lv_mild:fluid_day2 
             0.05112613

# plm_htn

## Fully adjusted

In [88]:
model <- update(covariates, ~ . 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day1_missing + fluid_day1
        + plm_htn_abnormal * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + plm_htn_abnormal:fluid_day1 + micu:plm_htn_abnormal + 
    micu:fluid_day1 + micu:plm_htn_abnormal:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5087  -0.7539  -0.5497   0.8478   2.4249  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.36792    0.26650  -8.885  < 2e-16 ***
age                               1.45765    0.36780   3.963 7.40e-05 ***
age_over_90                       1.09263    0.22001   4.966 6.82e-07 ***
gender                           -0.06823    0.10593  -0.644 0.519549    
bmi                               0.16807    0.13177   1.275 0.202139    
bmi_missing                 

(Intercept)                              age 
                      0.09367581                       4.29584392 
                     age_over_90                           gender 
                      2.98210527                       0.93404954 
                             bmi                      bmi_missing 
                      1.18301399                       1.44999812 
                       mech_vent                              CHF 
                      1.47485262                       0.97728305 
                            mdrd               creatinine_missing 
                      1.03140017                       0.95664373 
                          apsiii                             micu 
                      1.91609170                       1.24252860 
                         on_vaso                  plm_htn_missing 
                      2.67921122                       1.07134279 
                plm_htn_abnormal               fluid_day1_missing 
                      1.75706194                       1.62336845 
                      fluid_day1      plm_htn_abnormal:fluid_day1 
                      0.84308839                       0.77304878 
           micu:plm_htn_abnormal                  micu:fluid_day1 
                      0.81326488                       1.05384930 
micu:plm_htn_abnormal:fluid_day1 
                      1.60340611

In [89]:
model <- update(covariates, ~ . 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day2_missing + fluid_day2
        + plm_htn_abnormal * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + plm_htn_abnormal:fluid_day2 + micu:plm_htn_abnormal + 
    micu:fluid_day2 + micu:plm_htn_abnormal:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4898  -0.7489  -0.5507   0.8461   2.4098  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.3703869  0.2678826  -8.849  < 2e-16 ***
age                               1.4038392  0.3673116   3.822 0.000132 ***
age_over_90                       1.0531649  0.2199967   4.787 1.69e-06 ***
gender                           -0.0720352  0.1059548  -0.680 0.496589    
bmi                               0.1853373  0.1630216   1.137 0.255585    
bmi_missing     

(Intercept)                              age 
                      0.09344456                       4.07079868 
                     age_over_90                           gender 
                      2.86670964                       0.93049813 
                             bmi                      bmi_missing 
                      1.20362435                       1.52204974 
                       mech_vent                              CHF 
                      1.47215286                       1.00055822 
                            mdrd               creatinine_missing 
                      1.03512113                       1.02714863 
                          apsiii                             micu 
                      1.83896660                       1.23390499 
                         on_vaso                  plm_htn_missing 
                      2.55030565                       1.09168443 
                plm_htn_abnormal               fluid_day2_missing 
                      1.72887295                       1.36808174 
                      fluid_day2      plm_htn_abnormal:fluid_day2 
                      0.91130612                       0.91445901 
           micu:plm_htn_abnormal                  micu:fluid_day2 
                      0.84381976                       1.10215940 
micu:plm_htn_abnormal:fluid_day2 
                      1.33677569

In [90]:
model <- update(covariates, ~ . 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day3_missing + fluid_day3
        + plm_htn_abnormal * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + plm_htn_abnormal:fluid_day3 + micu:plm_htn_abnormal + 
    micu:fluid_day3 + micu:plm_htn_abnormal:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4969  -0.7541  -0.5477   0.8371   2.3966  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.39875    0.26916  -8.912  < 2e-16 ***
age                               1.34390    0.36709   3.661 0.000251 ***
age_over_90                       1.03512    0.22015   4.702 2.58e-06 ***
gender                           -0.07405    0.10604  -0.698 0.484975    
bmi                               0.20206    0.19358   1.044 0.296570    
bmi_missing                 

(Intercept)                              age 
                       0.0908317                        3.8339688 
                     age_over_90                           gender 
                       2.8154508                        0.9286232 
                             bmi                      bmi_missing 
                       1.2239187                        1.5861177 
                       mech_vent                              CHF 
                       1.5106487                        1.0161725 
                            mdrd               creatinine_missing 
                       1.0343697                        1.0468482 
                          apsiii                             micu 
                       1.8155055                        1.2273364 
                         on_vaso                  plm_htn_missing 
                       2.4935018                        1.0973683 
                plm_htn_abnormal               fluid_day3_missing 
                       1.6866699                        1.2461442 
                      fluid_day3      plm_htn_abnormal:fluid_day3 
                       0.7822136                        1.2138396 
           micu:plm_htn_abnormal                  micu:fluid_day3 
                       0.8756232                        1.4292865 
micu:plm_htn_abnormal:fluid_day3 
                       0.9824085

## Unadjusted

In [91]:
model <- glm(outcome ~ 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day1_missing + fluid_day1
        + plm_htn_abnormal * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ +plm_htn_missing + plm_htn_abnormal + 
    fluid_day1_missing + fluid_day1 + plm_htn_abnormal * fluid_day1 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2875  -0.8221  -0.7285   1.3193   2.0940  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.56313    0.21281  -7.345 2.06e-13 ***
plm_htn_missing                   0.17639    0.16037   1.100   0.2714    
plm_htn_abnormal                  0.57070    0.27920   2.044   0.0409 *  
fluid_day1_missing               -0.18002    0.22161  -0.812   0.4166    
fluid_day1                        0.21932    0.16263   1.349   0.1775    
micu                              0.37571    0.19198   1.957   0.0503 .  
plm_htn_abnormal:fluid_day1      -0.10126    0.24783  -0.409   0.6828    
plm_htn_abnormal:micu            -0.11437    0.27322  -0.419   0.6755    
fluid_day

(Intercept)                  plm_htn_missing 
                       0.2094793                        1.1929090 
                plm_htn_abnormal               fluid_day1_missing 
                       1.7695139                        0.8352505 
                      fluid_day1                             micu 
                       1.2452243                        1.4560276 
     plm_htn_abnormal:fluid_day1            plm_htn_abnormal:micu 
                       0.9036952                        0.8919316 
                 fluid_day1:micu plm_htn_abnormal:fluid_day1:micu 
                       1.0712624                        1.2882868

In [92]:
model <- glm(outcome ~ 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day2_missing + fluid_day2
        + plm_htn_abnormal * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ +plm_htn_missing + plm_htn_abnormal + 
    fluid_day2_missing + fluid_day2 + plm_htn_abnormal * fluid_day2 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2357  -0.8271  -0.7118   1.3232   2.1149  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.57588    0.21546  -7.314 2.59e-13 ***
plm_htn_missing                   0.16465    0.16071   1.025   0.3056    
plm_htn_abnormal                  0.55385    0.28353   1.953   0.0508 .  
fluid_day2_missing               -0.05181    0.14408  -0.360   0.7192    
fluid_day2                        0.29321    0.17824   1.645   0.1000 .  
micu                              0.38403    0.19491   1.970   0.0488 *  
plm_htn_abnormal:fluid_day2      -0.05261    0.26272  -0.200   0.8413    
plm_htn_abnormal:micu            -0.09036    0.27783  -0.325   0.7450    
fluid_day

(Intercept)                  plm_htn_missing 
                       0.2068256                        1.1789759 
                plm_htn_abnormal               fluid_day2_missing 
                       1.7399412                        0.9495137 
                      fluid_day2                             micu 
                       1.3407249                        1.4681914 
     plm_htn_abnormal:fluid_day2            plm_htn_abnormal:micu 
                       0.9487522                        0.9136004 
                 fluid_day2:micu plm_htn_abnormal:fluid_day2:micu 
                       1.0638024                        1.2096283

In [93]:
model <- glm(outcome ~ 
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day3_missing + fluid_day3
        + plm_htn_abnormal * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ +plm_htn_missing + plm_htn_abnormal + 
    fluid_day3_missing + fluid_day3 + plm_htn_abnormal * fluid_day3 * 
    micu, family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2422  -0.8558  -0.7080   1.2847   1.9767  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.53050    0.21244  -7.204 5.83e-13 ***
plm_htn_missing                   0.17573    0.16068   1.094   0.2741    
plm_htn_abnormal                  0.55242    0.27789   1.988   0.0468 *  
fluid_day3_missing               -0.09508    0.10754  -0.884   0.3766    
fluid_day3                        0.07657    0.19000   0.403   0.6869    
micu                              0.34898    0.19028   1.834   0.0666 .  
plm_htn_abnormal:fluid_day3       0.08604    0.28510   0.302   0.7628    
plm_htn_abnormal:micu            -0.07958    0.27204  -0.293   0.7699    
fluid_day

(Intercept)                  plm_htn_missing 
                       0.2164273                        1.1921102 
                plm_htn_abnormal               fluid_day3_missing 
                       1.7374557                        0.9092966 
                      fluid_day3                             micu 
                       1.0795804                        1.4176228 
     plm_htn_abnormal:fluid_day3            plm_htn_abnormal:micu 
                       1.0898514                        0.9234995 
                 fluid_day3:micu plm_htn_abnormal:fluid_day3:micu 
                       1.3798573                        1.0487129

## Model selection

In [39]:
model <- update(covariates, ~ .
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * plm_htn_abnormal * fluid_day1
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * plm_htn_abnormal * fluid_day1)
summary(select2)

Start:  AIC=2263.97
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1

                              Df Deviance    AIC
+ micu:fluid_day1              1   2226.4 2262.4
+ plm_htn_abnormal:fluid_day1  1   2227.9 2263.9
<none>                             2230.0 2264.0
+ micu:plm_htn_abnormal        1   2229.7 2265.7

Step:  AIC=2262.42
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1

                              Df Deviance    AIC
<none>                             2226.4 2262.4
+ plm_htn_abnormal:fluid_day1  1   2224.8 2262.8
+ micu:plm_htn_abnormal        1   2226.1 2264.1


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2229.972,2263.972
+ micu:fluid_day1,-1,3.550394,2151,2226.421,2262.421



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4208  -0.7551  -0.5473   0.8596   2.3810  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.29391    0.23773  -9.649  < 2e-16 ***
age                 1.44231    0.36682   3.932 8.43e-05 ***
age_over_90         1.09244    0.21957   4.975 6.51e-07 ***
gender             -0.06310    0.10571  -0.597 0.550566    
bmi                 0.16998    0.13539   1.255 0.209305    
bmi_missing         0.39173    0.19218   2.038 0.041513 *  
mech_vent           0.38461    0.11244   3.421 0.000625 ***
CHF                -0.02412    0.11359  -0.212 0.831850    
mdrd                0.0337


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1 + micu:plm_htn_abnormal + plm_htn_abnormal:fluid_day1 + 
    micu:plm_htn_abnormal:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5087  -0.7539  -0.5497   0.8478   2.4249  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.36792    0.26650  -8.885  < 2e-16 ***
age                               1.45765    0.36780   3.963 7.40e-05 ***
age_over_90                       1.09263    0.22001   4.966 6.82e-07 ***
gender                           -0.06823    0.10593  -0.644 0.519549    
bmi                               0.16807    0.13177   1.275 0.202139    
bmi_missing                 

In [40]:
model <- update(covariates, ~ .
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * plm_htn_abnormal * fluid_day2
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * plm_htn_abnormal * fluid_day2)
summary(select2)

Start:  AIC=2263.01
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2

                              Df Deviance    AIC
+ micu:fluid_day2              1   2226.4 2262.4
+ plm_htn_abnormal:fluid_day2  1   2226.9 2262.9
<none>                             2229.0 2263.0
+ micu:plm_htn_abnormal        1   2228.8 2264.8

Step:  AIC=2262.42
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + micu:fluid_day2

                              Df Deviance    AIC
<none>                             2226.4 2262.4
+ plm_htn_abnormal:fluid_day2  1   2224.6 2262.6
+ micu:plm_htn_abnormal        1   2226.2 2264.2


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2229.013,2263.013
+ micu:fluid_day2,-1,2.589993,2151,2226.423,2262.423



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + micu:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4203  -0.7533  -0.5508   0.8428   2.3790  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.31555    0.23972  -9.659  < 2e-16 ***
age                 1.39889    0.36647   3.817 0.000135 ***
age_over_90         1.05894    0.21966   4.821 1.43e-06 ***
gender             -0.06954    0.10580  -0.657 0.511006    
bmi                 0.18864    0.16762   1.125 0.260399    
bmi_missing         0.43430    0.19164   2.266 0.023435 *  
mech_vent           0.38983    0.11339   3.438 0.000586 ***
CHF                -0.00289    0.11362  -0.025 0.979711    
mdrd                0.0358


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day2_missing + 
    fluid_day2 + micu:fluid_day2 + micu:plm_htn_abnormal + plm_htn_abnormal:fluid_day2 + 
    micu:plm_htn_abnormal:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4898  -0.7489  -0.5507   0.8461   2.4098  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.3703869  0.2678826  -8.849  < 2e-16 ***
age                               1.4038392  0.3673116   3.822 0.000132 ***
age_over_90                       1.0531649  0.2199967   4.787 1.69e-06 ***
gender                           -0.0720352  0.1059548  -0.680 0.496589    
bmi                               0.1853373  0.1630216   1.137 0.255585    
bmi_missing     

In [41]:
model <- update(covariates, ~ .
        + plm_htn_missing + plm_htn_abnormal
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * plm_htn_abnormal * fluid_day3
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * plm_htn_abnormal * fluid_day3)
summary(select2)

Start:  AIC=2260.32
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3

                              Df Deviance    AIC
+ micu:fluid_day3              1   2221.5 2257.5
+ plm_htn_abnormal:fluid_day3  1   2223.9 2259.9
<none>                             2226.3 2260.3
+ micu:plm_htn_abnormal        1   2226.1 2262.1

Step:  AIC=2257.49
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3

                              Df Deviance    AIC
+ plm_htn_abnormal:fluid_day3  1   2219.4 2257.4
<none>                             2221.5 2257.5
+ micu:plm_htn_abnormal        1   2221.3 2259.3

Step:  AIC=2257.45
outcome ~ age + age_over_90 + gender + bmi + bmi_

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2226.323,2260.323
+ micu:fluid_day3,-1,4.834336,2151,2221.489,2257.489
+ plm_htn_abnormal:fluid_day3,-1,2.040925,2150,2219.448,2257.448



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3 + plm_htn_abnormal:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5017  -0.7522  -0.5462   0.8466   2.3758  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.34613    0.24218  -9.687  < 2e-16 ***
age                          1.34983    0.36677   3.680 0.000233 ***
age_over_90                  1.03339    0.22016   4.694 2.68e-06 ***
gender                      -0.07333    0.10604  -0.692 0.489222    
bmi                          0.20284    0.19480   1.041 0.297755    
bmi_missing                  0.45941    0.19233   2.389 0.016913 *  
mech_vent                    0.41239    0.11539   3.5


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_missing + plm_htn_abnormal + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3 + plm_htn_abnormal:fluid_day3 + 
    micu:plm_htn_abnormal + micu:plm_htn_abnormal:fluid_day3, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4969  -0.7541  -0.5477   0.8371   2.3966  

Coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -2.39875    0.26916  -8.912  < 2e-16 ***
age                               1.34390    0.36709   3.661 0.000251 ***
age_over_90                       1.03512    0.22015   4.702 2.58e-06 ***
gender                           -0.07405    0.10604  -0.698 0.484975    
bmi                               0.20206    0.19358   1.044 0.296570    
bmi_missing                 

# RV cavity

## Fully adjusted

In [43]:
model <- update(covariates, ~ . 
        + rvcav_missing + rvcav_dilated
        + fluid_day1_missing + fluid_day1
        + rvcav_dilated * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + rvcav_dilated:fluid_day1 + micu:rvcav_dilated + 
    micu:fluid_day1 + micu:rvcav_dilated:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3068  -0.7563  -0.5566   0.8612   2.3715  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.108006   0.208610 -10.105  < 2e-16 ***
age                            1.663051   0.362311   4.590 4.43e-06 ***
age_over_90                    1.214069   0.216175   5.616 1.95e-08 ***
gender                        -0.035949   0.105205  -0.342 0.732574    
bmi                            0.160800   0.124419   1.292 0.196214    
bmi_missing                    0.346196   0.190295   1

In [44]:
model <- update(covariates, ~ . 
        + rvcav_missing + rvcav_dilated
        + fluid_day2_missing + fluid_day2
        + rvcav_dilated * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated:fluid_day2 + micu:rvcav_dilated + 
    micu:fluid_day2 + micu:rvcav_dilated:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3125  -0.7588  -0.5579   0.8569   2.5966  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.12203    0.21010 -10.100  < 2e-16 ***
age                            1.63580    0.36200   4.519 6.22e-06 ***
age_over_90                    1.18303    0.21594   5.478 4.29e-08 ***
gender                        -0.03456    0.10531  -0.328 0.742782    
bmi                            0.17436    0.14495   1.203 0.229007    
bmi_missing                    0.38345    0.18973   2.021 0.

In [45]:
model <- update(covariates, ~ . 
        + rvcav_missing + rvcav_dilated
        + fluid_day3_missing + fluid_day3
        + rvcav_dilated * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + rvcav_dilated:fluid_day3 + micu:rvcav_dilated + 
    micu:fluid_day3 + micu:rvcav_dilated:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3442  -0.7565  -0.5612   0.8538   2.3445  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.15969    0.21314 -10.133  < 2e-16 ***
age                            1.58583    0.36145   4.387 1.15e-05 ***
age_over_90                    1.16287    0.21610   5.381 7.40e-08 ***
gender                        -0.03495    0.10535  -0.332 0.740069    
bmi                            0.18880    0.16930   1.115 0.264793    
bmi_missing                    0.41212    0.18990   2.170 0.

## Unadjusted

In [46]:
model <- glm(outcome ~ 
        + rvcav_missing + rvcav_dilated
        + fluid_day1_missing + fluid_day1
        + rvcav_dilated * fluid_day1 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + rvcav_dilated * fluid_day1 * micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3722  -0.8168  -0.7319   1.3095   2.0078  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -1.23397    0.13659  -9.034   <2e-16 ***
rvcav_missing                  0.28025    0.14339   1.954   0.0506 .  
rvcav_dilated                 -0.37780    0.39952  -0.946   0.3443    
fluid_day1_missing            -0.17680    0.22129  -0.799   0.4243    
fluid_day1                     0.17099    0.12595   1.358   0.1746    
micu                           0.25573    0.14640   1.747   0.0807 .  
rvcav_dilated:fluid_day1       0.02499    0.45790   0.055   0.9565    
rvcav_dilated:micu             0.60710    0.41811   1.452   0.1465    
fluid_day1:micu                0.12938    0.

In [47]:
model <- glm(outcome ~ 
        + rvcav_missing + rvcav_dilated
        + fluid_day2_missing + fluid_day2
        + rvcav_dilated * fluid_day2 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated * fluid_day2 * micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2685  -0.8221  -0.7179   1.3244   2.1897  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -1.25175    0.13873  -9.023   <2e-16 ***
rvcav_missing                  0.26811    0.14375   1.865   0.0622 .  
rvcav_dilated                 -0.47510    0.43166  -1.101   0.2711    
fluid_day2_missing            -0.04226    0.14378  -0.294   0.7688    
fluid_day2                     0.24610    0.13665   1.801   0.0717 .  
micu                           0.26874    0.14843   1.811   0.0702 .  
rvcav_dilated:fluid_day2       0.27216    0.45091   0.604   0.5461    
rvcav_dilated:micu             0.70868    0.44897   1.578   0.1145    
fluid_day2:micu                0.12093    0.

In [48]:
model <- glm(outcome ~ 
        + rvcav_missing + rvcav_dilated
        + fluid_day3_missing + fluid_day3
        + rvcav_dilated * fluid_day3 * micu
    , 
    family = binomial(link = 'logit'), 
    data = X)
summary(model)


Call:
glm(formula = outcome ~ +rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + rvcav_dilated * fluid_day3 * micu, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2557  -0.8087  -0.7322   1.3028   1.9857  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -1.21300    0.13727  -8.837   <2e-16 ***
rvcav_missing                  0.29128    0.14372   2.027   0.0427 *  
rvcav_dilated                 -0.44156    0.41700  -1.059   0.2896    
fluid_day3_missing            -0.08352    0.10733  -0.778   0.4365    
fluid_day3                     0.09568    0.14732   0.649   0.5160    
micu                           0.23764    0.14594   1.628   0.1034    
rvcav_dilated:fluid_day3       0.32704    0.50458   0.648   0.5169    
rvcav_dilated:micu             0.71178    0.43450   1.638   0.1014    
fluid_day3:micu                0.32857    0.

## Model selection

In [49]:
model <- update(covariates, ~ .
        + rvcav_missing + rvcav_dilated
        + fluid_day1_missing + fluid_day1
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * rvcav_dilated * fluid_day1
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * rvcav_dilated * fluid_day1)
summary(select2)

Start:  AIC=2273.53
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day1_missing + fluid_day1

                           Df Deviance    AIC
+ micu:fluid_day1           1   2236.2 2272.2
+ rvcav_dilated:fluid_day1  1   2236.6 2272.6
<none>                          2239.5 2273.5
+ micu:rvcav_dilated        1   2237.9 2273.9

Step:  AIC=2272.24
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day1_missing + fluid_day1 + 
    micu:fluid_day1

                           Df Deviance    AIC
+ rvcav_dilated:fluid_day1  1   2234.0 2272.0
<none>                          2236.2 2272.2
+ micu:rvcav_dilated        1   2234.7 2272.7

Step:  AIC=2272.01
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + cr

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2239.530,2273.530
+ micu:fluid_day1,-1,3.286192,2151,2236.244,2272.244
+ rvcav_dilated:fluid_day1,-1,2.231707,2150,2234.012,2272.012



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1 + rvcav_dilated:fluid_day1, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3143  -0.7580  -0.5573   0.8610   2.3385  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.17407    0.20315 -10.702  < 2e-16 ***
age                       1.66172    0.36215   4.588 4.46e-06 ***
age_over_90               1.21810    0.21608   5.637 1.73e-08 ***
gender                   -0.03605    0.10514  -0.343 0.731695    
bmi                       0.16117    0.12553   1.284 0.199165    
bmi_missing               0.34817    0.19034   1.829 0.067378 .  
mech_vent                 0.37525    0.11197   3.351 0.000804 ***
CHF             


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day1_missing + 
    fluid_day1 + micu:fluid_day1 + rvcav_dilated:fluid_day1 + 
    micu:rvcav_dilated + micu:rvcav_dilated:fluid_day1, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3068  -0.7563  -0.5566   0.8612   2.3715  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.108006   0.208610 -10.105  < 2e-16 ***
age                            1.663051   0.362311   4.590 4.43e-06 ***
age_over_90                    1.214069   0.216175   5.616 1.95e-08 ***
gender                        -0.035949   0.105205  -0.342 0.732574    
bmi                            0.160800   0.124419   1.292 0.196214    
bmi_missing                    0.346196   0.190295   1

In [50]:
model <- update(covariates, ~ .
        + rvcav_missing + rvcav_dilated
        + fluid_day2_missing + fluid_day2
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * rvcav_dilated * fluid_day2
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * rvcav_dilated * fluid_day2)
summary(select2)

Start:  AIC=2272.92
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day2_missing + fluid_day2

                           Df Deviance    AIC
+ rvcav_dilated:fluid_day2  1   2235.7 2271.7
+ micu:fluid_day2           1   2236.6 2272.6
<none>                          2238.9 2272.9
+ micu:rvcav_dilated        1   2237.3 2273.3

Step:  AIC=2271.74
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day2_missing + fluid_day2 + 
    rvcav_dilated:fluid_day2

                     Df Deviance    AIC
<none>                    2235.7 2271.7
+ micu:rvcav_dilated  1   2233.8 2271.8
+ micu:fluid_day2     1   2234.0 2272.0


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2238.920,2272.920
+ rvcav_dilated:fluid_day2,-1,3.184631,2151,2235.736,2271.736



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2872  -0.7565  -0.5586   0.8734   2.3651  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.22153    0.20494 -10.840  < 2e-16 ***
age                       1.61878    0.36149   4.478 7.53e-06 ***
age_over_90               1.18078    0.21568   5.475 4.38e-08 ***
gender                   -0.03498    0.10507  -0.333 0.739205    
bmi                       0.17430    0.14642   1.190 0.233902    
bmi_missing               0.38170    0.18986   2.010 0.044387 *  
mech_vent                 0.38582    0.11294   3.416 0.000635 ***
CHF                       0.02734 


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day2_missing + 
    fluid_day2 + rvcav_dilated:fluid_day2 + micu:rvcav_dilated + 
    micu:fluid_day2 + micu:rvcav_dilated:fluid_day2, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3125  -0.7588  -0.5579   0.8569   2.5966  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.12203    0.21010 -10.100  < 2e-16 ***
age                            1.63580    0.36200   4.519 6.22e-06 ***
age_over_90                    1.18303    0.21594   5.478 4.29e-08 ***
gender                        -0.03456    0.10531  -0.328 0.742782    
bmi                            0.17436    0.14495   1.203 0.229007    
bmi_missing                    0.38345    0.18973   2.021 0.

In [51]:
model <- update(covariates, ~ .
        + rvcav_missing + rvcav_dilated
        + fluid_day3_missing + fluid_day3
    , 
    family = binomial(link = 'logit'), 
    data = X)
scope <- formula(update(model, ~.
    + micu * rvcav_dilated * fluid_day3
    ))
select <- step(model, direction = 'forward', scope = scope)
select$anova
summary(select)

select2 <- update(select, ~ . + micu * rvcav_dilated * fluid_day3)
summary(select2)

Start:  AIC=2270.53
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day3_missing + fluid_day3

                           Df Deviance    AIC
+ micu:fluid_day3           1   2232.0 2268.0
+ rvcav_dilated:fluid_day3  1   2234.3 2270.3
<none>                          2236.5 2270.5
+ micu:rvcav_dilated        1   2234.9 2270.9

Step:  AIC=2267.99
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    rvcav_missing + rvcav_dilated + fluid_day3_missing + fluid_day3 + 
    micu:fluid_day3

                           Df Deviance    AIC
<none>                          2232.0 2268.0
+ rvcav_dilated:fluid_day3  1   2230.4 2268.4
+ micu:rvcav_dilated        1   2230.5 2268.5


Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2152,2236.533,2270.533
+ micu:fluid_day3,-1,4.545434,2151,2231.988,2267.988



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3683  -0.7570  -0.5613   0.8494   2.3152  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.21899    0.20755 -10.691  < 2e-16 ***
age                 1.56976    0.36099   4.348 1.37e-05 ***
age_over_90         1.16763    0.21598   5.406 6.44e-08 ***
gender             -0.03738    0.10518  -0.355 0.722291    
bmi                 0.19005    0.17202   1.105 0.269250    
bmi_missing         0.41474    0.19011   2.182 0.029144 *  
mech_vent           0.40447    0.11475   3.525 0.000424 ***
CHF                 0.04191    0.11388   0.368 0.712864    
mdrd                0.02680    


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + rvcav_missing + rvcav_dilated + fluid_day3_missing + 
    fluid_day3 + micu:fluid_day3 + micu:rvcav_dilated + rvcav_dilated:fluid_day3 + 
    micu:rvcav_dilated:fluid_day3, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3442  -0.7565  -0.5612   0.8538   2.3445  

Coefficients:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   -2.15969    0.21314 -10.133  < 2e-16 ***
age                            1.58583    0.36145   4.387 1.15e-05 ***
age_over_90                    1.16287    0.21610   5.381 7.40e-08 ***
gender                        -0.03495    0.10535  -0.332 0.740069    
bmi                            0.18880    0.16930   1.115 0.264793    
bmi_missing                    0.41212    0.18990   2.170 0.

# All echo variables

## Model selection

In [94]:
model <- update(covariates, ~ . + 
    + fluid_day1_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day1 * lv_hyper 
    + micu * fluid_day1 * lv_mild
    + micu * fluid_day1 * lv_depressed
    + micu * fluid_day1 * lv_severe
    + micu * fluid_day1 * plm_htn_abnormal
    + micu * fluid_day1 * rvcav_dilated
    )
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day1_missing + lv_missing + plm_htn_missing + 
    rvcav_missing + fluid_day1 + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_abnormal + rvcav_dilated + micu:fluid_day1 + 
    micu:lv_hyper + fluid_day1:lv_hyper + micu:lv_mild + fluid_day1:lv_mild + 
    micu:lv_depressed + fluid_day1:lv_depressed + micu:lv_severe + 
    fluid_day1:lv_severe + micu:plm_htn_abnormal + fluid_day1:plm_htn_abnormal + 
    micu:rvcav_dilated + fluid_day1:rvcav_dilated + micu:fluid_day1:lv_hyper + 
    micu:fluid_day1:lv_mild + micu:fluid_day1:lv_depressed + 
    micu:fluid_day1:lv_severe + micu:fluid_day1:plm_htn_abnormal + 
    micu:fluid_day1:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3826  -0.7529  -0.5314   0.8364   2.4472  

Coeffi

(Intercept)                              age 
                       0.0858007                        4.5780309 
                     age_over_90                           gender 
                       3.0152865                        0.9169631 
                             bmi                      bmi_missing 
                       1.1976216                        1.4370133 
                       mech_vent                              CHF 
                       1.4633430                        0.9410146 
                            mdrd               creatinine_missing 
                       1.0290174                        0.9193414 
                          apsiii                             micu 
                       1.9132835                        1.3041563 
                         on_vaso               fluid_day1_missing 
                       2.6202514                        1.5855061 
                      lv_missing                  plm_htn_missing 
                       0.9195984                        1.0253217 
                   rvcav_missing                       fluid_day1 
                       1.1607500                        0.7814584 
                        lv_hyper                          lv_mild 
                       2.0853393                        0.5451649 
                    lv_depressed                        lv_severe 
                       1.4676107                        2.5998605 
                plm_htn_abnormal                    rvcav_dilated 
                       1.8930339                        0.5282038 
                 micu:fluid_day1                    micu:lv_hyper 
                       1.0847551                        0.7693062 
             fluid_day1:lv_hyper                     micu:lv_mild 
                       0.8628674                        1.2657224 
              fluid_day1:lv_mild                micu:lv_depressed 
                       4.3201056                        0.8428365 
         fluid_day1:lv_depressed                   micu:lv_severe 
                       0.9651266                        0.6090225 
            fluid_day1:lv_severe            micu:plm_htn_abnormal 
                       1.0217746                        0.7327856 
     fluid_day1:plm_htn_abnormal               micu:rvcav_dilated 
                       0.8123109                        1.9249696 
        fluid_day1:rvcav_dilated         micu:fluid_day1:lv_hyper 
                       1.3092093                        1.2505541 
         micu:fluid_day1:lv_mild     micu:fluid_day1:lv_depressed 
                       0.1640197                        1.1454994 
       micu:fluid_day1:lv_severe micu:fluid_day1:plm_htn_abnormal 
                       1.2155802                        1.4989295 
   micu:fluid_day1:rvcav_dilated 
                       0.8898285

In [105]:
model <- update(covariates, ~. + 
    + fluid_day1_missing + fluid_day1 + 
    + lv_missing + lv_hyper + lv_mild + lv_depressed + lv_severe
    + plm_htn_missing + plm_htn_abnormal
    + rvcav_missing + rvcav_dilated
)
summary(model)
scope <- formula(update(covariates, ~. + 
    + fluid_day1_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day1 * lv_hyper 
    + micu * fluid_day1 * lv_mild
    + micu * fluid_day1 * lv_depressed
    + micu * fluid_day1 * lv_severe
    + micu * fluid_day1 * plm_htn_abnormal
    + micu * fluid_day1 * rvcav_dilated
    ))
select <- step(covariates, direction = 'forward', scope = scope)
select$anova
summary(select)
exp(coef(select))

select2 <- update(select, ~ . + micu * fluid_day1 * lv_mild)
summary(select2)
exp(coef(select2))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day1_missing + fluid_day1 + lv_missing + 
    lv_hyper + lv_mild + lv_depressed + lv_severe + plm_htn_missing + 
    plm_htn_abnormal + rvcav_missing + rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3757  -0.7518  -0.5455   0.8553   2.4902  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.37996    0.24200  -9.835  < 2e-16 ***
age                 1.46333    0.37043   3.950 7.80e-05 ***
age_over_90         1.09279    0.22130   4.938 7.89e-07 ***
gender             -0.07077    0.10691  -0.662  0.50800    
bmi                 0.17613    0.14302   1.232  0.21812    
bmi_missing         0.38903    0.19349   2.011  0.04437 *  
mech_vent           0.38321    0.11306   3.389  0.00070 ***
CHF   

Start:  AIC=2270.08
outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso

                     Df Deviance    AIC
+ plm_htn_abnormal    1   2234.6 2262.6
+ lv_hyper            1   2236.2 2264.2
+ lv_severe           1   2238.3 2266.3
+ plm_htn_missing     1   2240.1 2268.1
+ fluid_day1_missing  1   2240.6 2268.6
+ lv_mild             1   2241.4 2269.4
<none>                    2244.1 2270.1
+ lv_missing          1   2243.1 2271.1
+ lv_depressed        1   2243.2 2271.2
+ fluid_day1          1   2243.5 2271.5
+ rvcav_missing       1   2243.9 2271.9
+ rvcav_dilated       1   2244.0 2272.0

Step:  AIC=2262.59
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_abnormal

                        Df Deviance    AIC
+ lv_hyper               1   2226.8 2256.8
+ lv_severe              1   2229.2 2259.2
+ fluid_day1_mis

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2156,2244.078,2270.078
+ plm_htn_abnormal,-1,9.489013,2155,2234.589,2262.589
+ lv_hyper,-1,7.813362,2154,2226.776,2256.776
+ lv_severe,-1,6.476098,2153,2220.300,2252.300
+ fluid_day1_missing,-1,3.194051,2152,2217.106,2251.106



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_abnormal + lv_hyper + lv_severe + fluid_day1_missing, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3464  -0.7486  -0.5478   0.8642   2.4150  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.34473    0.20619 -11.372  < 2e-16 ***
age                 1.44336    0.36756   3.927 8.61e-05 ***
age_over_90         1.08271    0.21987   4.924 8.47e-07 ***
gender             -0.06617    0.10614  -0.623 0.533011    
bmi                 0.18191    0.15116   1.203 0.228801    
bmi_missing         0.41713    0.19178   2.175 0.029625 *  
mech_vent           0.38380    0.11181   3.433 0.000598 ***
CHF                -0.03818    0.11488  -0.332 0.739655    
mdrd                0.03581    0.05774   0.620 0.535128  

(Intercept)                age        age_over_90             gender 
        0.09587335         4.23490098         2.95268344         0.93597473 
               bmi        bmi_missing          mech_vent                CHF 
        1.19950671         1.51759884         1.46785864         0.96254269 
              mdrd creatinine_missing             apsiii               micu 
        1.03645711         1.02862759         1.85417752         1.18508121 
           on_vaso   plm_htn_abnormal           lv_hyper          lv_severe 
        2.57069688         1.39514177         1.70927264         1.64381329 
fluid_day1_missing 
        1.54960743


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_abnormal + lv_hyper + lv_severe + fluid_day1_missing + 
    fluid_day1 + lv_mild + micu:fluid_day1 + micu:lv_mild + fluid_day1:lv_mild + 
    micu:fluid_day1:lv_mild, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4132  -0.7510  -0.5379   0.8384   2.4621  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.31335    0.20889 -11.075  < 2e-16 ***
age                      1.50808    0.36985   4.078 4.55e-05 ***
age_over_90              1.10418    0.22125   4.991 6.02e-07 ***
gender                  -0.07119    0.10656  -0.668 0.504063    
bmi                      0.18910    0.16658   1.135 0.256315    
bmi_missing              0.40582    0.19328   2.100 0.035756 *  
mech_vent                0.38

(Intercept)                     age             age_over_90 
             0.09892905              4.51805771              3.01674327 
                 gender                     bmi             bmi_missing 
             0.93128085              1.20815758              1.50053947 
              mech_vent                     CHF                    mdrd 
             1.47252173              0.95587544              1.03457141 
     creatinine_missing                  apsiii                    micu 
             0.98240666              1.91977100              1.13041530 
                on_vaso        plm_htn_abnormal                lv_hyper 
             2.60230275              1.41427547              1.67779234 
              lv_severe      fluid_day1_missing              fluid_day1 
             1.58862176              1.61668707              0.71756920 
                lv_mild         micu:fluid_day1            micu:lv_mild 
             0.51647185              1.39347150              1.34624565 
     fluid_day1:lv_mild micu:fluid_day1:lv_mild 
             4.48925441              0.15579738

In [96]:
model <- update(covariates, ~ . + 
    + fluid_day2_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day2 * lv_hyper 
    + micu * fluid_day2 * lv_mild
    + micu * fluid_day2 * lv_depressed
    + micu * fluid_day2 * lv_severe
    + micu * fluid_day2 * plm_htn_abnormal
    + micu * fluid_day2 * rvcav_dilated
    )
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day2_missing + lv_missing + plm_htn_missing + 
    rvcav_missing + fluid_day2 + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_abnormal + rvcav_dilated + micu:fluid_day2 + 
    micu:lv_hyper + fluid_day2:lv_hyper + micu:lv_mild + fluid_day2:lv_mild + 
    micu:lv_depressed + fluid_day2:lv_depressed + micu:lv_severe + 
    fluid_day2:lv_severe + micu:plm_htn_abnormal + fluid_day2:plm_htn_abnormal + 
    micu:rvcav_dilated + fluid_day2:rvcav_dilated + micu:fluid_day2:lv_hyper + 
    micu:fluid_day2:lv_mild + micu:fluid_day2:lv_depressed + 
    micu:fluid_day2:lv_severe + micu:fluid_day2:plm_htn_abnormal + 
    micu:fluid_day2:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3835  -0.7516  -0.5292   0.8159   2.4236  

Coeffi

(Intercept)                              age 
                      0.08320048                       4.59397436 
                     age_over_90                           gender 
                      2.94364052                       0.91855933 
                             bmi                      bmi_missing 
                      1.22317784                       1.49745968 
                       mech_vent                              CHF 
                      1.46455542                       0.94676557 
                            mdrd               creatinine_missing 
                      1.03413615                       0.96648675 
                          apsiii                             micu 
                      1.83012266                       1.29154818 
                         on_vaso               fluid_day2_missing 
                      2.51392547                       1.36109500 
                      lv_missing                  plm_htn_missing 
                      0.91071477                       1.06366494 
                   rvcav_missing                       fluid_day2 
                      1.14479512                       0.76821133 
                        lv_hyper                          lv_mild 
                      2.01455199                       0.28980478 
                    lv_depressed                        lv_severe 
                      1.21067155                       2.87461396 
                plm_htn_abnormal                    rvcav_dilated 
                      1.95641021                       0.43549127 
                 micu:fluid_day2                    micu:lv_hyper 
                      1.27172142                       0.80795580 
             fluid_day2:lv_hyper                     micu:lv_mild 
                      1.20734768                       2.39731945 
              fluid_day2:lv_mild                micu:lv_depressed 
                     11.27174979                       1.06946128 
         fluid_day2:lv_depressed                   micu:lv_severe 
                      1.75666535                       0.57436733 
            fluid_day2:lv_severe            micu:plm_htn_abnormal 
                      1.18633141                       0.73928956 
     fluid_day2:plm_htn_abnormal               micu:rvcav_dilated 
                      0.92215747                       2.29750582 
        fluid_day2:rvcav_dilated         micu:fluid_day2:lv_hyper 
                      1.85416508                       0.77010261 
         micu:fluid_day2:lv_mild     micu:fluid_day2:lv_depressed 
                      0.04276080                       0.69849806 
       micu:fluid_day2:lv_severe micu:fluid_day2:plm_htn_abnormal 
                      1.20938507                       1.31372279 
   micu:fluid_day2:rvcav_dilated 
                      0.61662939

In [98]:
scope <- formula(update(covariates, ~. + 
    + fluid_day2_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day2 * lv_hyper 
    + micu * fluid_day2 * lv_mild
    + micu * fluid_day2 * lv_depressed
    + micu * fluid_day2 * lv_severe
    + micu * fluid_day2 * plm_htn_abnormal
    + micu * fluid_day2 * rvcav_dilated
    ))
select <- step(covariates, direction = 'forward', scope = scope)
select$anova
summary(select)
exp(coef(select))

select2 <- update(select, ~ . + micu * fluid_day2 * lv_mild)
summary(select2)
exp(coef(select2))

Start:  AIC=2270.08
outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso

                     Df Deviance    AIC
+ plm_htn_abnormal    1   2234.6 2262.6
+ lv_hyper            1   2236.2 2264.2
+ lv_severe           1   2238.3 2266.3
+ plm_htn_missing     1   2240.1 2268.1
+ fluid_day2_missing  1   2240.4 2268.4
+ lv_mild             1   2241.4 2269.4
<none>                    2244.1 2270.1
+ fluid_day2          1   2242.7 2270.7
+ lv_missing          1   2243.1 2271.1
+ lv_depressed        1   2243.2 2271.2
+ rvcav_missing       1   2243.9 2271.9
+ rvcav_dilated       1   2244.0 2272.0

Step:  AIC=2262.59
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_abnormal

                        Df Deviance    AIC
+ lv_hyper               1   2226.8 2256.8
+ lv_severe              1   2229.2 2259.2
+ fluid_day2_mis

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2156,2244.078,2270.078
+ plm_htn_abnormal,-1,9.489013,2155,2234.589,2262.589
+ lv_hyper,-1,7.813362,2154,2226.776,2256.776
+ lv_severe,-1,6.476098,2153,2220.300,2252.300
+ fluid_day2_missing,-1,3.511283,2152,2216.789,2250.789



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_abnormal + lv_hyper + lv_severe + fluid_day2_missing, 
    family = binomial(link = "logit"), data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3314  -0.7507  -0.5498   0.8536   2.4147  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -2.37123    0.20759 -11.422  < 2e-16 ***
age                 1.43380    0.36756   3.901 9.58e-05 ***
age_over_90         1.06789    0.21977   4.859 1.18e-06 ***
gender             -0.06830    0.10620  -0.643 0.520138    
bmi                 0.18604    0.15647   1.189 0.234457    
bmi_missing         0.42404    0.19158   2.213 0.026875 *  
mech_vent           0.40638    0.11323   3.589 0.000332 ***
CHF                -0.03193    0.11480  -0.278 0.780929    
mdrd                0.04013    0.05779   0.694 0.487413  

(Intercept)                age        age_over_90             gender 
        0.09336538         4.19462077         2.90922108         0.93398095 
               bmi        bmi_missing          mech_vent                CHF 
        1.20446757         1.52811949         1.50136704         0.96857878 
              mdrd creatinine_missing             apsiii               micu 
        1.04094628         1.05184638         1.85168065         1.18335107 
           on_vaso   plm_htn_abnormal           lv_hyper          lv_severe 
        2.56165711         1.39856310         1.71745514         1.63655737 
fluid_day2_missing 
        1.35941005


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_abnormal + lv_hyper + lv_severe + fluid_day2_missing + 
    fluid_day2 + lv_mild + micu:fluid_day2 + micu:lv_mild + fluid_day2:lv_mild + 
    micu:fluid_day2:lv_mild, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4135  -0.7475  -0.5356   0.8219   2.3701  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.32259    0.21032 -11.043  < 2e-16 ***
age                      1.48323    0.37013   4.007 6.14e-05 ***
age_over_90              1.07900    0.22199   4.861 1.17e-06 ***
gender                  -0.07928    0.10683  -0.742 0.458013    
bmi                      0.22537    0.24830   0.908 0.364065    
bmi_missing              0.43922    0.19283   2.278 0.022742 *  
mech_vent                0.39

(Intercept)                     age             age_over_90 
             0.09801918              4.40717095              2.94173973 
                 gender                     bmi             bmi_missing 
             0.92377775              1.25278474              1.55149217 
              mech_vent                     CHF                    mdrd 
             1.48019585              0.96767801              1.03663985 
     creatinine_missing                  apsiii                    micu 
             1.05496325              1.83499217              1.13338454 
                on_vaso        plm_htn_abnormal                lv_hyper 
             2.49363953              1.42171228              1.65449461 
              lv_severe      fluid_day2_missing              fluid_day2 
             1.62525858              1.37395822              0.83247062 
                lv_mild         micu:fluid_day2            micu:lv_mild 
             0.26844940              1.38673073              2.59143982 
     fluid_day2:lv_mild micu:fluid_day2:lv_mild 
            10.33445533              0.04551971

In [99]:
model <- update(covariates, ~ . + 
    + fluid_day3_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day3 * lv_hyper 
    + micu * fluid_day3 * lv_mild
    + micu * fluid_day3 * lv_depressed
    + micu * fluid_day3 * lv_severe
    + micu * fluid_day3 * plm_htn_abnormal
    + micu * fluid_day3 * rvcav_dilated
    )
summary(model)
exp(coef(model))


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + fluid_day3_missing + lv_missing + plm_htn_missing + 
    rvcav_missing + fluid_day3 + lv_hyper + lv_mild + lv_depressed + 
    lv_severe + plm_htn_abnormal + rvcav_dilated + micu:fluid_day3 + 
    micu:lv_hyper + fluid_day3:lv_hyper + micu:lv_mild + fluid_day3:lv_mild + 
    micu:lv_depressed + fluid_day3:lv_depressed + micu:lv_severe + 
    fluid_day3:lv_severe + micu:plm_htn_abnormal + fluid_day3:plm_htn_abnormal + 
    micu:rvcav_dilated + fluid_day3:rvcav_dilated + micu:fluid_day3:lv_hyper + 
    micu:fluid_day3:lv_mild + micu:fluid_day3:lv_depressed + 
    micu:fluid_day3:lv_severe + micu:fluid_day3:plm_htn_abnormal + 
    micu:fluid_day3:rvcav_dilated, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4035  -0.7470  -0.5362   0.8145   2.5102  

Coeffi

(Intercept)                              age 
                      0.08222638                       4.36661601 
                     age_over_90                           gender 
                      2.96512918                       0.92549028 
                             bmi                      bmi_missing 
                      1.25119967                       1.55516429 
                       mech_vent                              CHF 
                      1.49921559                       0.97550714 
                            mdrd               creatinine_missing 
                      1.03778375                       1.07638920 
                          apsiii                             micu 
                      1.80758004                       1.25401665 
                         on_vaso               fluid_day3_missing 
                      2.45395772                       1.22328959 
                      lv_missing                  plm_htn_missing 
                      0.92380559                       1.06972741 
                   rvcav_missing                       fluid_day3 
                      1.15037448                       0.66623170 
                        lv_hyper                          lv_mild 
                      1.92221571                       0.67853169 
                    lv_depressed                        lv_severe 
                      1.38673797                       2.76561145 
                plm_htn_abnormal                    rvcav_dilated 
                      1.78178958                       0.47080689 
                 micu:fluid_day3                    micu:lv_hyper 
                      1.67355948                       0.83629994 
             fluid_day3:lv_hyper                     micu:lv_mild 
                      1.42898131                       1.02986947 
              fluid_day3:lv_mild                micu:lv_depressed 
                      4.90016810                       0.93550359 
         fluid_day3:lv_depressed                   micu:lv_severe 
                      1.66813313                       0.57676368 
            fluid_day3:lv_severe            micu:plm_htn_abnormal 
                      1.92814820                       0.81182756 
     fluid_day3:plm_htn_abnormal               micu:rvcav_dilated 
                      1.13397198                       2.13971747 
        fluid_day3:rvcav_dilated         micu:fluid_day3:lv_hyper 
                      1.61026499                       0.69231337 
         micu:fluid_day3:lv_mild     micu:fluid_day3:lv_depressed 
                      0.08820896                       0.83943839 
       micu:fluid_day3:lv_severe micu:fluid_day3:plm_htn_abnormal 
                      0.51799649                       1.02907371 
   micu:fluid_day3:rvcav_dilated 
                      0.72081980

In [101]:
scope <- formula(update(covariates, ~. + 
    + fluid_day3_missing + lv_missing + plm_htn_missing + rvcav_missing
    + micu * fluid_day3 * lv_hyper 
    + micu * fluid_day3 * lv_mild
    + micu * fluid_day3 * lv_depressed
    + micu * fluid_day3 * lv_severe
    + micu * fluid_day3 * plm_htn_abnormal
    + micu * fluid_day3 * rvcav_dilated
    ))
select <- step(covariates, direction = 'forward', scope = scope)
select$anova
summary(select)
exp(coef(select))

select2 <- update(select, ~ . + micu * fluid_day3 * lv_mild)
summary(select2)
exp(coef(select2))

Start:  AIC=2270.08
outcome ~ +age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso

                     Df Deviance    AIC
+ plm_htn_abnormal    1   2234.6 2262.6
+ lv_hyper            1   2236.2 2264.2
+ lv_severe           1   2238.3 2266.3
+ fluid_day3          1   2239.8 2267.8
+ plm_htn_missing     1   2240.1 2268.1
+ fluid_day3_missing  1   2241.2 2269.2
+ lv_mild             1   2241.4 2269.4
<none>                    2244.1 2270.1
+ lv_missing          1   2243.1 2271.1
+ lv_depressed        1   2243.2 2271.2
+ rvcav_missing       1   2243.9 2271.9
+ rvcav_dilated       1   2244.0 2272.0

Step:  AIC=2262.59
outcome ~ age + age_over_90 + gender + bmi + bmi_missing + mech_vent + 
    CHF + mdrd + creatinine_missing + apsiii + micu + on_vaso + 
    plm_htn_abnormal

                        Df Deviance    AIC
+ lv_hyper               1   2226.8 2256.8
+ lv_severe              1   2229.2 2259.2
+ fluid_day3    

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,2156,2244.078,2270.078
+ plm_htn_abnormal,-1,9.489013,2155,2234.589,2262.589
+ lv_hyper,-1,7.813362,2154,2226.776,2256.776
+ lv_severe,-1,6.476098,2153,2220.300,2252.300
+ fluid_day3,-1,4.536860,2152,2215.763,2249.763
+ micu:fluid_day3,-1,4.721611,2151,2211.041,2247.041
+ fluid_day3_missing,-1,2.692358,2150,2208.349,2246.349
+ fluid_day3:plm_htn_abnormal,-1,2.146844,2149,2206.202,2246.202
+ lv_depressed,-1,2.126449,2148,2204.076,2246.076
+ fluid_day3:lv_depressed,-1,2.192749,2147,2201.883,2245.883



Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_abnormal + lv_hyper + lv_severe + fluid_day3 + 
    fluid_day3_missing + lv_depressed + micu:fluid_day3 + plm_htn_abnormal:fluid_day3 + 
    fluid_day3:lv_depressed, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4226  -0.7412  -0.5435   0.8309   2.3766  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.37691    0.21141 -11.243  < 2e-16 ***
age                          1.40978    0.36918   3.819 0.000134 ***
age_over_90                  1.05474    0.22157   4.760 1.93e-06 ***
gender                      -0.08184    0.10676  -0.767 0.443315    
bmi                          0.21613    0.22020   0.982 0.326337    
bmi_missing                  0.46781    0.19278   2.427 0.015238 *  
me

(Intercept)                         age 
                 0.09283706                  4.09504233 
                age_over_90                      gender 
                 2.87121631                  0.92141588 
                        bmi                 bmi_missing 
                 1.24125907                  1.59648826 
                  mech_vent                         CHF 
                 1.49832037                  0.97516530 
                       mdrd          creatinine_missing 
                 1.03841398                  1.08108605 
                     apsiii                        micu 
                 1.79879628                  1.16956099 
                    on_vaso            plm_htn_abnormal 
                 2.45224466                  1.39819956 
                   lv_hyper                   lv_severe 
                 1.73031768                  1.70948702 
                 fluid_day3          fluid_day3_missing 
                 0.77268708                  1.20347243 
               lv_depressed             micu:fluid_day3 
                 1.35538899                  1.40816005 
plm_htn_abnormal:fluid_day3     fluid_day3:lv_depressed 
                 1.19918174                  1.48937031


Call:
glm(formula = outcome ~ age + age_over_90 + gender + bmi + bmi_missing + 
    mech_vent + CHF + mdrd + creatinine_missing + apsiii + micu + 
    on_vaso + plm_htn_abnormal + lv_hyper + lv_severe + fluid_day3 + 
    fluid_day3_missing + lv_depressed + lv_mild + micu:fluid_day3 + 
    plm_htn_abnormal:fluid_day3 + fluid_day3:lv_depressed + micu:lv_mild + 
    fluid_day3:lv_mild + micu:fluid_day3:lv_mild, family = binomial(link = "logit"), 
    data = X)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4619  -0.7431  -0.5384   0.8184   2.5062  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.37658    0.21466 -11.071  < 2e-16 ***
age                          1.46373    0.37107   3.945 7.99e-05 ***
age_over_90                  1.08602    0.22288   4.873 1.10e-06 ***
gender                      -0.07689    0.10721  -0.717 0.473283    
bmi                          0.23638    0.27387   0.863 0.38807

(Intercept)                         age 
                 0.09286759                  4.32206924 
                age_over_90                      gender 
                 2.96247212                  0.92599509 
                        bmi                 bmi_missing 
                 1.26665225                  1.56957613 
                  mech_vent                         CHF 
                 1.49852525                  0.96957183 
                       mdrd          creatinine_missing 
                 1.03847677                  1.10018313 
                     apsiii                        micu 
                 1.80899146                  1.16302898 
                    on_vaso            plm_htn_abnormal 
                 2.44789503                  1.39471258 
                   lv_hyper                   lv_severe 
                 1.69392426                  1.69101842 
                 fluid_day3          fluid_day3_missing 
                 0.73232134                  1.21405416 
               lv_depressed                     lv_mild 
                 1.34148980                  0.62754466 
            micu:fluid_day3 plm_htn_abnormal:fluid_day3 
                 1.54502797                  1.20338574 
    fluid_day3:lv_depressed                micu:lv_mild 
                 1.44884529                  1.15007425 
         fluid_day3:lv_mild     micu:fluid_day3:lv_mild 
                 4.68934151                  0.09322480